In [18]:
import numpy as np
import pandas as pd
import LVQ
from random import randrange

In [19]:
train = pd.read_csv(r"SD-2X_rocktype.csv")
x_train = train.iloc[:, 0:6].values
y_train = train.iloc[:, 6].values
y_train[y_train==4] = 0

In [20]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
x_train = minmax.fit_transform(x_train)

In [21]:
test = pd.read_csv(r"SD-3X_rocktype.csv")
x_test = test.iloc[:, 0:6].values
y_test = test.iloc[:, 6].values
y_test[y_test==4] = 0
x_test = minmax.transform(x_test)

In [22]:
# Create a random subsample from the dataset with replacement
def subsample(x, y, ratio):
    sample_x = list()
    sample_y = list()
    n_sample = round(len(x) * ratio)
    while len(sample_x) < n_sample:
        index = randrange(len(x))
        sample_x.append(x[index])
        sample_y.append(y[index])
    return [np.asarray(sample_x), np.asarray(sample_y)]

In [23]:
# Bootstrap Aggregation Algorithm
def bagging(x_train, y_train, x_test, sample_size, n_LVQ):
    trees = list()
    for i in range(n_LVQ):
        sample = subsample(x_train, y_train, sample_size)
        lvq = LVQ.LVQ(sample[0], sample[1], n_classes=4, p_vectors=[], n_neurons=5)
        y_pred = [lvq.predict(instance) for instance in x_test]
        trees.append(y_pred)
    return(np.asarray(trees))

In [24]:
# Make a prediction with a list of bagged trees
def bagging_predict(predictions, n_row, n_LVQ):
#     for i in range(n_row):
#         votes = [predictions[j][i]]
    return [np.bincount([predictions[j][i] for j in range(n_LVQ)]).argmax() for i in range(n_row)]

In [25]:
a = bagging(x_train, y_train, x_test, sample_size=0.3, n_LVQ=5)
y_pred = bagging_predict(a, n_row=len(a[0]), n_LVQ=5)
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 1,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,


In [26]:
from sklearn.metrics import classification_report
print (classification_report(y_test,y_pred,target_names=['0', '1', '2', '3']))

             precision    recall  f1-score   support

          0       0.98      0.60      0.74       331
          1       0.95      0.95      0.95        95
          2       0.51      0.92      0.66        26
          3       0.58      0.89      0.70       205

avg / total       0.83      0.75      0.76       657

